## ***beyond the DAG*** - patterns in more complicated workflows

### Concepts
- **Flow of Flows**: parent orchestrator patterns and subflows

- **Breaking the DAG**: changing the control flow on the fly

## Extending the `nike_flow` from earlier
Now imagine we have many URLs pointing to Nike shoes that we like, and we want to save all the shoes in our budget. 

We can create an instance of our `nike_flow` for each `url`, all wrapped in a new function we call `parent_flow`:


```python
@flow
def parent_flow(urls: List[str], budget: int) -> None:
    for url in urls:
        nike_flow(url, budget)
```


In Prefect language, this is what we call a [subflow](https://orion-docs.prefect.io/concepts/flows/#subflows) pattern. 

Using such a pattern, we can dynamically change the execution of the `parent_flow` based on the results we get from each `nike_flow` instance (subflow) ***during runtime***. 

This what we mean by *"breaking the DAG"* since representing workflows as DAGs requires knowing the execution plan ***before runtime***.



## our flow of flows

Imports!

In [2]:
from bs4 import BeautifulSoup
from prefect import flow, task
from typing import List
import requests
import re

Let's use our old friend `find_nice_price` from earlier:

In [3]:
@task(retries=3, retry_delay_seconds=10)
def find_nike_price(url: str) -> int:
    k = requests.get(url).text
    soup = BeautifulSoup(k,'html.parser')
    price_string = soup.find('div', {"class":"product-price"}).text.replace(' ','')
    price = re.search('[0-9]+', price_string).group(0)
    return int(price)

In [4]:
@task
def inBudget(price: int, budget: int) -> bool:
    return price <= budget

In [5]:
@task
def save(url: str, to: str = 'shoes_in_my_budget.md') -> None:
    with open(to, 'w') as f:
        link = f"[{url.split('/t/')[-1]}]({url})\n\n"
        f.write(link)

In [6]:
@flow
def nike_flow(url: str, budget: int) -> None:
    price = find_nike_price(url)
    
    if inBudget(price, budget).result():
        save(url)

In [7]:
@flow
def parent_flow(urls: List[str], budget: int) -> None:
    for url in urls:
        nike_flow(url, budget)

In [8]:
urls = [
    "https://www.nike.com/t/air-max-270-womens-shoes-Pgb94t/AH6789-601",
    "https://www.nike.com/t/air-max-terrascape-90-mens-shoes-R6r8hB/DH2973-100",
    "https://www.nike.com/t/pegasus-trail-3-gore-tex-mens-running-shoes-HG005k/DR0137-200"
]

budget = 150

parent_flow(urls, budget)

02:15:16.191 | INFO    | prefect.engine - Created flow run 'icy-jackrabbit' for flow 'parent-flow'
02:15:16.192 | INFO    | Flow run 'icy-jackrabbit' - Using task runner 'ConcurrentTaskRunner'
02:15:16.293 | INFO    | Flow run 'icy-jackrabbit' - Created subflow run 'nimble-fossa' for flow 'nike-flow'
02:15:16.367 | INFO    | Flow run 'nimble-fossa' - Created task run 'find_nike_price-3ad0ad07-0' for task 'find_nike_price'
02:15:16.424 | INFO    | Flow run 'nimble-fossa' - Created task run 'inBudget-d36ec0d3-0' for task 'inBudget'
/Users/nate/opt/miniconda3/envs/chess/lib/python3.7/site-packages/prefect/client.py:1196: UserWarning: No default storage has been set on the server. Using temporary local storage for results.
  "No default storage has been set on the server. "
02:15:17.405 | INFO    | Task run 'find_nike_price-3ad0ad07-0' - Finished in state Completed(None)
02:15:17.443 | INFO    | Task run 'inBudget-d36ec0d3-0' - Finished in state Completed(None)
02:15:17.471 | INFO    | Flo

Completed(message='All states completed.', type=COMPLETED, result=[Completed(message='All states completed.', type=COMPLETED, result=[Completed(message=None, type=COMPLETED, result=160, task_run_id=50a24187-d889-4a28-8dc6-1cafd21a1dd5), Completed(message=None, type=COMPLETED, result=False, task_run_id=c8c64e5f-4fed-45af-a1c6-cc477c68e1ee)], task_run_id=85cdfe25-59ff-4dd5-9e02-d81918ac9a3f), Completed(message='All states completed.', type=COMPLETED, result=[Completed(message=None, type=COMPLETED, result=140, task_run_id=b3ac2c04-48ce-4438-970a-f6d84db96d9e), Completed(message=None, type=COMPLETED, result=True, task_run_id=7d26217b-1aa6-4b0a-b768-f1aeba4c4ccf), Completed(message=None, type=COMPLETED, result=None, task_run_id=0436c654-9711-4b17-8d55-04661f0558ba)], task_run_id=1c8abb5a-54ff-495c-be55-69ba6648f640), Completed(message='All states completed.', type=COMPLETED, result=[Completed(message=None, type=COMPLETED, result=160, task_run_id=ff7b1323-cafb-4cfa-9bf5-239fc8d8e909), Comple

## What did it do?
Let's look at the output found in the [shoes_in_my_budget](shoes_in_my_budget.md) markdown file...


We see the links to the URLs pointing to shoes that have a `price` that is less than our `budget`.

### Benefits of using subflow patterns when applicable

- Ability to orchestrate many instances of a complex, stateful workflow without duplicating work

- Maintain visibility into each of the subflow processes via the UI

<hr>

### Discussion: What else might we want to use a subflow pattern for?

#### Q&A
